### Goal : Generate the configurations for ImageMagick

##### Libraries

In [29]:
import os, time 
import numpy as np
import pandas as pd
from scipy import stats

##### Runtime configs

In [30]:
cols = ["-limit memory", "-posterize", "-gaussian-blur", "-limit thread", "-quality"]
dico = dict({"-limit memory" : ["2mb", "10mb", "100mb", "1GiB", "2GiB"""],
             "-posterize" : ["2","4","10","20"],
             "-gaussian-blur" : ["1","11", "21"],
             "-limit thread" : ["1", "2", "5", "12"],
             "-quality" : ["1", "50", "100"]})

In [31]:
df = pd.DataFrame({})

nb_conf = 1000
nb_conf_final = 100

for d in dico.keys():
    df[d] = np.random.randint(0,len(dico[d]),nb_conf)

df = df.drop_duplicates()[0:nb_conf_final].reset_index().drop(["index"], axis=1)
df

,-limit memory,-posterize,-gaussian-blur,-limit thread,-quality
0,4,2,2,0,0
1,0,0,1,2,0
2,1,2,2,2,2
3,3,3,1,0,1
4,2,2,0,1,0
...,...,...,...,...,...
95,3,0,1,1,2
96,3,0,0,3,2
97,0,3,2,1,2
98,0,3,2,1,1


In [32]:
df.to_csv('runtime_options.csv')

In [33]:
for j in range(nb_conf_final):
    with open("./scripts/"+str(j)+".sh","w") as f:
        f.write('#!/bin/bash\nnumb="'+str(j))
        f.write('"\nlogfilename="./logs/$numb.log"\ninputlocation="$1"\n')
        f.write("# execute the configuration and measure its time\n")
        vals = df.iloc[j]
        cmd_line = "{ time ./ImageMagick/utilities/magick ./images/$inputlocation.JPEG "
        list_conf = ""
        for i in range(len(vals)):
            cmd_line+=cols[i]+" "+dico[cols[i]][vals[i]]+" "
            list_conf+=str(dico[cols[i]][vals[i]])+","
            val_config = str(dico[cols[i]][vals[i]])
            list_conf+=str(int(val_config == ""))+","*(i!=(len(vals)-1))
        cmd_line+= " ./test.png; } 2> $logfilename\n"
        f.write(cmd_line)
        f.write("# extract compiled program size \n")
        f.write("size=`ls -lrt ./test.png | awk '{print $5}'`\n")
        f.write("# analyze log to extract relevant timing information and CPU usage\n")
        f.write("""time=`grep "user" $logfilename | sed 's/user/,/;s/elapsed/,/;s/system/,/' | cut -d "%" -f 1`\n""")
        f.write("# clean\n")
        f.write("eval `rm test.png`\n")
        f.write('''csvLine="$numb,''')
        f.write(list_conf+'''"\n''')
        f.write('''csvLine="$csvLine,$size,$time"\n''')
        f.write('''echo $csvLine''')